In [9]:
from google.cloud import storage
import os
import vertexai
from vertexai.generative_models import GenerativeModel, Part, FinishReason
import vertexai.preview.generative_models as generative_models

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/home/pan/Documents/2025/github/video-understanding/resources/ai-2c-gemini.json"

In [10]:
def upload_to_gcs(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    # Upload the file
    blob.upload_from_filename(source_file_name)

    print(f"File {source_file_name} uploaded to {destination_blob_name}.")
    return {
        "name": blob.name,
        "size": blob.size,
        "content_type": blob.content_type,
        "public_url": blob.public_url
    }

    

In [11]:
bucket_name = "ai-2c-video"  # The name of your GCS bucket
source_file_name = "/home/pan/Documents/2025/github/video-understanding/test_resources/audios/1.m4a"
destination_blob_name = "audios/1.m4a"

upload_to_gcs(bucket_name, source_file_name, destination_blob_name)


File /home/pan/Documents/2025/github/video-understanding/test_resources/audios/1.m4a uploaded to audios/1.m4a.


{'name': 'audios/1.m4a',
 'size': 3572290,
 'content_type': 'audio/mp4',
 'public_url': 'https://storage.googleapis.com/ai-2c-video/audios/1.m4a'}

In [17]:
vertexai.init(project="ai-2c-gemini", location="us-central1")

model = GenerativeModel(
    "gemini-2.0-flash",
)

generation_config = {
    "max_output_tokens": 8192,
    "temperature": 1,
    "top_p": 0.95,
}

In [18]:
language = "Chinese"
prompt = """
# 角色设定：
你是一位影视解说专家，擅长根据剧情描述视频的画面和故事生成一段有趣且吸引人的解说文案。你特别熟悉 tiktok/抖音 风格的影视解说文案创作。

# 任务目标：
1.	根据给定的剧情描述，详细描述视频画面并展开叙述，尤其是对重要画面进行细致刻画。
2.	生成风格符合 tiktok/抖音 的影视解说文案，使其节奏快、内容抓人。
3.	最终结果以 JSON 格式输出，字段包含：
  • "picture"：画面描述
  • "timestamp"：时间戳（表示画面出现的时间-画面结束的时间）
  • "narration"：对应的解说文案

# 输入示例：
```text
在一个黑暗的小巷中，主角缓慢走进，四周静谧无声，只有远处隐隐传来猫的叫声。突然，背后出现一个神秘的身影。
```

# 输出格式：
```json
[
    {
        "picture": "黑暗的小巷中，主角缓慢走进，四周静谧无声，远处有模糊的猫叫声。",
        "timestamp": "00:00-00:17",
        "narration": "昏暗的小巷里，他独自前行，空气中透着一丝不安，隐约中能听到远处的猫叫声。 "
    },
    {
        "picture": "主角背后突然出现一个神秘的身影，气氛骤然紧张。",
        "timestamp": "00:17-00:39",
        "narration": "就在他以为安全时，一个身影悄无声息地出现在他身后，危险一步步逼近！ "
    }
    ...
]
```
# 提示：
  - 生成的解说文案应简洁有力，符合短视频平台用户的偏好。
  - 叙述中应有强烈的代入感和悬念，以吸引观众持续观看。
  - 文案语言为：%s
""" % (language)

In [19]:
prompt_2 = f'''
    你是一个有用的视频助手，擅长对视频的内容进行归纳总结；可以按照下面的步骤进行：
    1. 首先理解视频的内容,可以从视频的字幕，人物的表情， 场景等细节来充分理解视频内容；
    2. 根据第一步的视频理解，将视频划分成几个场景(视频只有一个场景就不用划分)；
    3. 然后给每个场景生成一个总结标题，标题能够概括这个场景；同时生成一个能够清晰描述这个场景的文案；
    4. 输出，输出格式使用json的格式进行输出，每个场景应该包含下面的字段：
        - title: 场景的标题，不超过10个字；
        - description: 场景的描述文案；
        - index: 场景的索引，从1开始；
        - timestamp: 时间戳（表示画面出现的时间-画面结束的时间）；

'''

In [23]:
prompt_3 = "这段音乐的情绪是什么？"

In [21]:
prompt_4 = """
    请用中文详细描述这个视频的内容，需要给出三种描述：详细描述、缩略描述和标题。
    详细描述(detailed)：首先按时间顺序，根据内容的显著变化如镜头切换、场景切换、主要动作变化、话题转换等，将视频划分为多个视频片段(segment_%d)，片段划分应根据视频总时长自动调整，不要平均时间切断，短视频（30秒以内）允许产生时长更短的更细粒度分段，至少3段，以镜头切换、人物出入场、关键动作的起止、场景内的焦点转移作为分段点，长视频（3分种以上）允许产生时长更长的更粗粒度分段，至多15段，以较大的叙事单元、完整的场景或持续时间较长且主题一致的动作序列作为整体。每个视频片段都要提供精确的开始时间点(start_time)和结束时间点(end_time)， 都以秒为单位，然后尽量详细但不过度推测地描述每个视频片段的主体对象、场景构成、动作行为、表情情绪、说话口型、人声音乐、色彩色调、光影效果、材质纹理、风格类型、美学质量、画面缺陷、镜头运动、场景转换等信息，注明显著特征和特殊细节（如文字、字幕、品牌标识等），分析可能的情感氛围或隐喻意义，描述应连贯、流畅，准确反映视频的进程，不同片段之间的人物和场景要有一致性描述——这点非常重要，不超过200字。
    缩略描述(thumbnail)：概括整个视频内容，包括主体行为/场景类型、最具辨识度的视觉特征、整体情绪基调等，不超过100个字。
    标题(title)：提供关于图像的关键信息，客观，信息量越大越好，不超过10个字。
    输出严格遵循json格式：
    {
        detailed: {
            segment_%d: {
                content: %s, 
                start_time: %d, 
                end_time: %d}
        }, 
        thumbnail: %s,
        title: %s, 
        total_time: %d
    }
    其中%d表示整数，%s表示不包含emoji和特殊符号标记的文字内容，请严格按照此结构输出，不要添加任何额外字符。仅输出有效的JSON格式，不要包含任何解释性文字、注释或代码块标记 (json ...)。请在输出前检查 JSON 的有效性。
"""

In [24]:
video_url = f"gs://{bucket_name}/{destination_blob_name}"
video_file = Part.from_uri(video_url, mime_type="audio/mp3")
res = model.generate_content([video_file, prompt_3])
print(res.text)

充满活力且充满趣味性。
